In [1]:
# just upload existing audience and see the difference of the size between the LR uploaded and this new direcet upload

import pandas as pd
import os
import numpy as np
import datetime
import glob

from facebookads.api import FacebookAdsApi
from facebookads.adobjects.adaccount import AdAccount
from facebookads.adobjects.customaudience import CustomAudience


In [3]:
BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBAHsJtDc00ZBtZAW2ye9CRdKsy9ivLlzvgIMsEkZAyZA3mRpuGBF8IvZAR9obyYZCdzfw455nzMEW76A5ZBZBS6Fft2nQsUSs8sZCcGDZBZCbVeiCq0LMvekWjnnDZCuglkNihTAZAWIWTpvzaZBeZCKYywrPl0ZAsoWul36cyQZDZD"

def creatfbaudience(file_path,ACCESS_TOKEN,accountid,description,audience_name):
    '''
    input:
    file_path: str, full_path, basename as the audience name on FB
        make sure the columns contains the "email_address_hash", use this col only
    
    ACCESS_TOKEN: personal token #app id not needed
    accountid: account_id starting with "act_", followed by the id str
    as Nov. 20, 2019, latest verision is 'v5.0'
    '''
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v6.0')
    myAccount = AdAccount(accountid)
    fields = []
    fbname=audience_name
    params = {
      'name': fbname,
      'subtype': 'CUSTOM',
      'description': description,
      'customer_file_source': 'USER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    
    df=pd.read_csv(file_path,dtype=str,)
    listlen = df.shape[0]
    requestUrl = 'https://graph.facebook.com/v5.0/'+ fbname + '/users'
    
    chunck_size=50000
    i_counter=0
    for df_chunck in pd.read_csv(file_path,chunksize=chunck_size,dtype=str,usecols=["email_address_hash"]):
        payload = {}
        payload['schema'] = ['EMAIL']
        df_chunck=df_chunck.rename(columns={"email_address_hash":"EMAIL"})
        df_chunck=df_chunck.drop_duplicates()
        payload_data=df_chunck['EMAIL'].apply(lambda x: [x]).tolist()

        payload['data'] = payload_data
        params = {}
        params ={'payload':payload} 
        CustomAudience(fbname).create_user(
                      fields=fields,
                      params=params,
        )
        i_counter+=1
        print(i_counter,datetime.datetime.now())
  

    return 'All done: '+str(datetime.datetime.now())

In [4]:
# 1 0222_P
'''
Size
55,000
Description
LR_LRData_BigLots_segmentnewsignups=NewReward022220P
Type
Custom Audience
Shared by
851984484996900
'''

# 2 0222_T
'''
Size
62,000
Description
LR_LRData_BigLots_segmentnewsignups=NewReward022220T
Type
Custom Audience
Shared by
851984484996900
'''

# 3 0222_S
'''
Size
25,000
Description
LR_LRData_BigLots_segmentnewsignups=NewReward022220S
Type
Custom Audience
Shared by
851984484996900
'''

# Total
'''
NewReward_022220_P 82,849
NewReward_022220_S 33,813
NewReward_022220_T 78,052
'''

'\nNewReward_022220_P 82,849\nNewReward_022220_S 33,813\nNewReward_022220_T 78,052\n'

In [5]:
# check the new upload to FB as test

list_new_signs_uploaded_to_LR=glob.glob("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/*.csv")
list_new_signs_uploaded_to_LR

['/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_P.csv',
 '/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_S.csv',
 '/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_T.csv']

In [6]:
for local_file in list_new_signs_uploaded_to_LR:

    creatfbaudience(file_path=local_file,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    description="test_no_use_JL_"+os.path.basename(local_file).split(".")[0],
                    audience_name="test_no_use_JL_"+os.path.basename(local_file).split(".")[0]
                   )

1 2020-03-03 10:48:37.816220
2 2020-03-03 10:48:42.423448
1 2020-03-03 10:48:49.538844
1 2020-03-03 10:48:54.190403
2 2020-03-03 10:48:59.401678
